In [ ]:



# =============================================================================
# Automatic Installation of Required Packages
# =============================================================================
import subprocess  # Module to run subprocess commands (e.g., pip installation)
import sys         # System-specific parameters and functions

# A list of required packages that will be installed if not already present.
required_packages = [
    'pandas', 'numpy', 'matplotlib', 'seaborn', 'scikit-learn', 'xgboost',
    'lightgbm', 'openpyxl', 'tensorflow', 'imblearn',
    'statsmodels', 'skrebate', 'umap-learn', 'boruta', 'shap', 'joblib', 'xlsxwriter'
]

# Loop through each package and try to import it; if the package is missing, install it.
for package in required_packages:
    try:
        # Replace hyphen with underscore when importing (if needed)
        __import__(package.replace('-', '_'))
    except ImportError:
        # Install the missing package using pip
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# =============================================================================
# Importing Required Libraries
# =============================================================================
import os                          # For file and path operations
import shutil                      # For file operations (e.g., copying files)
import logging                     # For logging error and info messages
from pathlib import Path           # For object-oriented filesystem paths
from datetime import datetime      # For handling date and time

import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical operations

import matplotlib.pyplot as plt    # Plotting library
import seaborn as sns              # Statistical data visualization
import traceback
# Importing specific models and functions from scikit-learn and other libraries

# Gaussian Process Classifier and Multi-layer Perceptron classifier from sklearn
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier

# Model selection and cross-validation tools
from sklearn.model_selection import (train_test_split, StratifiedKFold,
                                     RandomizedSearchCV, GridSearchCV)

# Metrics for evaluation
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix)

# Imputation and scaling/preprocessing functions
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (MinMaxScaler, StandardScaler, RobustScaler, Normalizer, MaxAbsScaler)

# Feature selection functions and methods
from sklearn.feature_selection import (SelectKBest, chi2, f_classif, mutual_info_classif,
                                       VarianceThreshold, RFE, SequentialFeatureSelector, SelectFdr,
                                       SelectFwe)

# Dimensionality reduction methods (feature extraction)
from sklearn.decomposition import (PCA, FastICA, TruncatedSVD, KernelPCA, FactorAnalysis, SparsePCA, NMF)

# Clustering and hierarchical grouping for features
from sklearn.cluster import FeatureAgglomeration

# Manifold learning techniques for non-linear dimension reduction
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS, SpectralEmbedding

# Importing classifiers from XGBoost and LightGBM libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Importing additional classifiers from scikit-learn
from sklearn.svm import SVC
from sklearn.linear_model import (LogisticRegression, Lasso, RidgeClassifier, ElasticNet,
                                  LogisticRegressionCV, SGDClassifier)
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier,
                              GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier,
                              HistGradientBoostingClassifier, StackingClassifier)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier

# Importing feature selection methods from additional libraries
from skrebate import ReliefF
from boruta import BorutaPy

# Import SHAP for model explainability
import shap

# Import oversampling technique from imbalanced-learn
from imblearn.over_sampling import ADASYN

# Other libraries
import joblib
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy.stats import norm  # Statistical distributions

# =============================================================================
# Global Settings and Parameters
# ==============================================================================

# List of training dataset file paths (absolute paths)
TRAINING_FILES = [
    r"Five fold cross Input"  # <-- Example: Replace with your real file path
]
# ➤ You can provide 1 to 3 training files
# ➤ Supported formats: .csv, .xlsx, .xls
# Optional: Custom folder to store result Excel files
RESULTS_FOLDER = r"Saving folder"
# ➤ Example: r"D:\MyResults"
# ➤ If None, results will be saved to a default 'res' folder

# Absolute path to external test dataset (CSV or Excel)
EXTERNAL_TEST_FILE = r"External input"

#EXTERNAL_TEST_FILE = None

#EXTERNAL_TEST_FILE = r"C:\Users\mahyar\OneDrive\Desktop\idh\final\data\external_test.csv"
# ➤ Must contain the same columns as training data
# ➤ Also requires 'PatientID' and 'Outcome' columns


# Choose analysis mode: 'Supervised' or 'Semi-Supervised'
ANALYSIS_MODE = 'Supervised'
# ➤ Options:
#     'Supervised'       = Use only rows with labeled Outcome
#     'Semi-Supervised'  = Fill missing Outcome values using predicted labels


# List of outcome classifiers to be used for outcome prediction
OUTCOME_CLASSIFIERS = ["Logistic_Regression"]
# ➤ Available classifier names (strings from INVOLVED_CLASSIFIERS):
#     - "Logistic_Regression"
#     - "Decision_Tree"
#     - "Random_Forest"
#     - "Support_Vector_Machine"
#     - "K_Nearest_Neighbors"
#     - "XGBoost"
#     - "LightGBM"
#     - "MLP_Classifier"
#     - "Gaussian_Naive_Bayes"
#     - etc.

# Choose test mode: 'External' or 'Internal'
TEST_MODE = 'External'  # ➤ If 'Internal', TEST_SIZE will be used


# Use the provided training files directly
dataset_files = TRAINING_FILES



RANDOM_SEED = 42
# ➤ Used to ensure reproducibility of results


N_FOLDS = 5
# ➤ Number of folds for Stratified K-Fold cross-validation
# ➤ Common values: 3, 5, 10


TEST_SIZE = 0.2
# ➤ Only used if splitting from training set (not relevant when external test is provided)


NOF = 10
# ➤ Number Of Features (NOF) to select
# ➤ Affects feature selection and extraction


SELECTED_CLASSES = [0, 1]
# ➤ List of valid class labels to keep from the Outcome column


CLASS_MAPPING = {0: 0, 1: 1}
# ➤ Maps each class name to a numeric value
# ➤ Make sure it matches the SELECTED_CLASSES order

# --------------------- #
# Class Selection Percentages
# --------------------- #
# Define the percentage of patients to select from each class before mapping
# Values should be between 0 and 1
CLASS_SELECTION_PERCENT = {
    0: 1.0,    # 100%
    1: 1.0      # 100%
}


SCALING_METHOD = 'MinMaxScaler'
# ➤ Options:
#     'MinMaxScaler'
#     'StandardScaler'
#     'RobustScaler'
#     'Normalizer'
#     'MaxAbsScaler'
# ➤ Set to any invalid value to skip scaling


IMPUTATION_STRATEGY = 'mean'
# ➤ Options:
#     'mean'           = replace missing values with column mean
#     'median'         = replace with column median
#     'most_frequent'  = replace with most common value


# Set logging level for debugging or tracking info
logging.getLogger().setLevel(logging.INFO)



# =============================================================================
# Validate Dataset Paths
# =============================================================================

# Check if all training files exist
for file_path in TRAINING_FILES:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"Training file '{file_path}' not found.")

# Check if external test file exists
if TEST_MODE == 'External':
    if not os.path.isfile(EXTERNAL_TEST_FILE):
        raise FileNotFoundError(f"External test file '{EXTERNAL_TEST_FILE}' not found.")

# Warn user if external file is provided but TEST_MODE is not 'External'
if TEST_MODE != 'External' and EXTERNAL_TEST_FILE and os.path.isfile(EXTERNAL_TEST_FILE):
    print("⚠️ WARNING: External test file is provided but TEST_MODE is set to 'Internal'. The file will be ignored.")




# =============================================================================
# Helper Functions
# =============================================================================

def load_dataframe(file_path):
    """
    Load a dataset from a CSV or Excel file into a pandas DataFrame.
    Automatically detects format from file extension.
    """
    if file_path.lower().endswith(".csv"):
        return pd.read_csv(file_path)
    elif file_path.lower().endswith((".xlsx", ".xls")):
        return pd.read_excel(file_path, engine="openpyxl")
    else:
        raise ValueError(f"Unsupported file type: {file_path}")


def show_class_distribution(before_df, after_df, outcome_col='Outcome'):
    print("\n📊 Class Distribution BEFORE Filtering:")
    before_counts = before_df[outcome_col].value_counts(dropna=False)
    before_percent = (before_counts / before_counts.sum()) * 100
    for cls in before_counts.index:
        print(f" - {cls}: {before_counts[cls]} samples ({before_percent[cls]:.1f}%)")

    print("\n📉 Class Distribution AFTER Filtering:")
    after_counts = after_df[outcome_col].value_counts(dropna=False)
    after_percent = (after_counts / after_counts.sum()) * 100
    for cls in after_counts.index:
        print(f" - {cls}: {after_counts[cls]} samples ({after_percent[cls]:.1f}%)")



def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None, **to_excel_kwargs):
    """
    Append a DataFrame to an existing Excel file.
    Creates the file if it does not exist, or appends to a specific sheet.
    """
    from openpyxl import load_workbook

    if not os.path.isfile(filename):
        df.to_excel(filename, sheet_name=sheet_name, index=False, **to_excel_kwargs)
        return

    try:
        writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='overlay')
        workbook = writer.book

        if startrow is None:
            if sheet_name in workbook.sheetnames:
                startrow = workbook[sheet_name].max_row
            else:
                startrow = 0

        df.to_excel(writer, sheet_name=sheet_name, startrow=startrow,
                    index=False, header=(startrow == 0), **to_excel_kwargs)
        writer.close()
    except Exception as e:
        logging.error(f"Error writing to Excel file {filename} in sheet {sheet_name}: {e}")


def compute_auc_and_specificity(model, X, y):
    """
    Compute AUC and Specificity metrics for the given model and input data.
    Uses predict_proba or decision_function to compute probabilities.
    """
    try:
        # If the model supports predict_proba, use it to get probability for the positive class.
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X)[:, 1]
        # If predict_proba is not available, try decision_function.
        elif hasattr(model, "decision_function"):
            probs = model.decision_function(X)
        else:
            probs = None
        # Compute AUC if probabilities are available.
        auc = roc_auc_score(y, probs) if probs is not None else np.nan
    except Exception as e:
        logging.error(f"Error computing AUC: {e}")
        auc = np.nan

    try:
        # Obtain the predicted class labels.
        y_pred = model.predict(X)
        # Calculate the confusion matrix values: True Negative, False Positive, False Negative, True Positive.
        tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
        # Calculate Specificity as TN/(TN+FP)
        specificity = tn / (tn + fp) if (tn + fp) != 0 else np.nan
    except Exception as e:
        logging.error(f"Error computing Specificity: {e}")
        specificity = np.nan

    return auc, specificity

def train_improved_autoencoder(X, encoding_dim, epochs=100, batch_size=32):
    """
    Sample function to train an improved autoencoder.
    In this version, it is not implemented and simply returns None.
    """
    logging.warning("train_improved_autoencoder is not implemented; returning None.")
    return None

def integrate_autoencoder_features(X, encoder, encoding_dim):
    """
    Integrate features extracted by an autoencoder with the original data.
    In this version, it is not implemented and the original data is returned.
    """
    logging.warning("integrate_autoencoder_features is not implemented; returning original data.")
    return X

# =============================================================================
# Feature Selection and Extraction Functions
# =============================================================================

def apply_chi_square(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using the Chi-Square test.
    Falls back to mutual_info_classif in case of an error.
    """
    try:
        # Select K best features using the chi2 score function.
        selector = SelectKBest(score_func=chi2, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector
    except Exception as e:
        logging.error(f"Error during Chi-Square feature selection: {e}")
        # Fall back using mutual information if an error occurs
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector

def apply_correlation_coefficient(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on correlation coefficients.
    Falls back to f_classif if an error occurs.
    """
    try:
        # Compute the correlation matrix of the training data (features only).
        corr_matrix = np.corrcoef(X_train, rowvar=False)
        # Obtain correlations of each feature with the target (assumed to be in the last column).
        target_corr = corr_matrix[-1][:-1]
        # Select the indices of the top absolute correlations.
        top_idx = np.argsort(np.abs(target_corr))[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in correlation-based feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector.get_support(indices=True)

def apply_mutual_information(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using Mutual Information.
    Falls back to Chi2 in case of an error.
    """
    try:
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector
    except Exception as e:
        logging.error(f"Error during Mutual Information feature selection: {e}")
        selector = SelectKBest(score_func=chi2, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector

def apply_variance_threshold(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using a variance threshold.
    Falls back to f_classif if any error occurs.
    """
    try:
        selector = VarianceThreshold()
        X_var = selector.fit_transform(X_train)
        variances = selector.variances_
        # Select features with the highest variances.
        top_idx = np.argsort(variances)[-num_features:]
        return X_var[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in Variance Threshold feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector.get_support(indices=True)

def apply_anova_f_test(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using the ANOVA F-test.
    Falls back to using mutual information in case of error.
    """
    try:
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector
    except Exception as e:
        logging.error(f"Error in ANOVA F-test: {e}")
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector

def apply_information_gain(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on Information Gain (functionally similar to Mutual Information).
    """
    try:
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector
    except Exception as e:
        logging.error(f"Error in Information Gain feature selection: {e}")
        selector = SelectKBest(score_func=chi2, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector

def apply_univariate_feature_selection(X_train, y_train, num_features, **kwargs):
    """
    Univariate feature selection using a specified scoring function.
    Defaults to mutual information if no scoring function is provided.
    Falls back to f_classif in case of an error.
    """
    try:
        score_func = kwargs.get('score_func', mutual_info_classif)
        selector = SelectKBest(score_func=score_func, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector
    except Exception as e:
        logging.error(f"Error in univariate feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector

def apply_lasso_fs(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using LASSO regression.
    Falls back to Chi2 if an error occurs.
    """
    try:
        alpha_ = kwargs.get('alpha', 0.01)
        lasso = Lasso(alpha=alpha_)
        lasso.fit(X_train, y_train)
        # Use the absolute coefficient values to rank features.
        coefs = np.abs(lasso.coef_)
        top_idx = np.argsort(coefs)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in LASSO feature selection: {e}")
        selector = SelectKBest(score_func=chi2, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector.get_support(indices=True)

def apply_rfe_fs(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using Recursive Feature Elimination (RFE).
    Falls back to f_classif-based selection if an error occurs.
    """
    try:
        estimator = LogisticRegression(max_iter=1000)
        rfe = RFE(estimator=estimator, n_features_to_select=num_features, step=1)
        rfe.fit(X_train, y_train)
        selected_idx = rfe.get_support(indices=True)
        return X_train[:, selected_idx], selected_idx
    except Exception as e:
        logging.error(f"Error in RFE feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector.get_support(indices=True)

def apply_elastic_net_fs(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using the Elastic Net method.
    Falls back to LASSO if an error occurs.
    """
    try:
        alpha_ = kwargs.get('alpha', 0.01)
        l1_ratio_ = kwargs.get('l1_ratio', 0.5)
        en = ElasticNet(alpha=alpha_, l1_ratio=l1_ratio_, random_state=42)
        en.fit(X_train, y_train)
        coefs = np.abs(en.coef_)
        top_idx = np.argsort(coefs)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in Elastic Net feature selection: {e}")
        lasso = Lasso(alpha=0.01)
        lasso.fit(X_train, y_train)
        coefs = np.abs(lasso.coef_)
        top_idx = np.argsort(coefs)[-num_features:]
        return X_train[:, top_idx], top_idx

def apply_sequential_feature_selector_func(X_train, y_train, num_features, **kwargs):
    """
    Sequential Feature Selector for stepwise feature selection.
    Falls back to RFE if any error occurs.
    """
    try:
        direction = kwargs.get('direction', 'forward')
        scoring = kwargs.get('scoring', 'accuracy')
        cv = kwargs.get('cv', 3)
        estimator = LogisticRegression(max_iter=1000, random_state=42)
        selector = SequentialFeatureSelector(estimator=estimator, n_features_to_select=num_features,
                                             direction=direction, scoring=scoring, cv=cv, n_jobs=-1)
        selector.fit(X_train, y_train)
        selected_idx = selector.get_support(indices=True)
        return X_train[:, selected_idx], selected_idx
    except Exception as e:
        logging.error(f"Error in sequential feature selection: {e}")
        rfe = RFE(estimator=LogisticRegression(max_iter=1000, random_state=42),
                  n_features_to_select=num_features)
        rfe.fit(X_train, y_train)
        selected_idx = rfe.get_support(indices=True)
        return X_train[:, selected_idx], selected_idx

def apply_select_fdr(X_train, y_train, num_features, **kwargs):
    """
    Feature selection with False Discovery Rate (FDR) control.
    Falls back to SelectFpr if an error occurs.
    """
    try:
        score_func_ = kwargs.get('score_func', f_classif)
        alpha_ = kwargs.get('alpha', 0.05)
        selector = SelectFdr(score_func=score_func_, alpha=alpha_)
        selector.fit(X_train, y_train)
        scores = selector.scores_
        top_idx = np.argsort(scores)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in FDR feature selection: {e}")
        from sklearn.feature_selection import SelectFpr
        selector = SelectFpr(score_func=f_classif, alpha=0.05)
        selector.fit(X_train, y_train)
        scores = selector.scores_
        top_idx = np.argsort(scores)[-num_features:]
        return X_train[:, top_idx], top_idx

def apply_select_fwe(X_train, y_train, num_features, **kwargs):
    """
    Feature selection with Family-Wise Error (FWE) control.
    Falls back to SelectFpr if an error occurs.
    """
    try:
        score_func_ = kwargs.get('score_func', f_classif)
        alpha_ = kwargs.get('alpha', 0.05)
        selector = SelectFwe(score_func=score_func_, alpha=alpha_)
        selector.fit(X_train, y_train)
        scores = selector.scores_
        top_idx = np.argsort(scores)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in FWE feature selection: {e}")
        from sklearn.feature_selection import SelectFpr
        selector = SelectFpr(score_func=f_classif, alpha=0.05)
        selector.fit(X_train, y_train)
        scores = selector.scores_
        top_idx = np.argsort(scores)[-num_features:]
        return X_train[:, top_idx], top_idx

def apply_feature_importance_rf(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on feature importances from a Random Forest model.
    Falls back to mutual_info_classif-based selection if an error occurs.
    """
    try:
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        importances = rf.feature_importances_
        top_idx = np.argsort(importances)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in Random Forest importance feature selection: {e}")
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector.get_support(indices=True)

def apply_permutation_importance_fs(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using permutation importance.
    Falls back to Random Forest based feature importance if an error occurs.
    """
    from sklearn.inspection import permutation_importance
    try:
        scoring = kwargs.get('scoring', 'accuracy')
        model = LogisticRegression(max_iter=1000, random_state=42)
        model.fit(X_train, y_train)
        result = permutation_importance(model, X_train, y_train,
                                        n_repeats=10, random_state=42, n_jobs=-1, scoring=scoring)
        importances = result.importances_mean
        top_idx = np.argsort(importances)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in permutation importance feature selection: {e}")
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        importances = rf.feature_importances_
        top_idx = np.argsort(importances)[-num_features:]
        return X_train[:, top_idx], top_idx

def apply_relief_f(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using the ReliefF algorithm.
    Falls back to f_classif based selection if an error occurs.
    """
    try:
        n_feat = kwargs.get('n_features_to_select', 10)
        relief = ReliefF(n_features_to_select=n_feat)
        relief.fit(X_train, y_train)
        top_idx = relief.top_features_[:num_features]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in ReliefF feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_selected = selector.fit_transform(X_train, y_train)
        return X_selected, selector.get_support(indices=True)

def apply_mutual_info_selection(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using mutual information.
    Falls back to f_classif in case an error occurs.
    """
    try:
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector
    except Exception as e:
        logging.error(f"Error in mutual information feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector

def apply_select_from_model_lr(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using Logistic Regression as a SelectFromModel method.
    Falls back to RFE if an error occurs.
    """
    from sklearn.feature_selection import SelectFromModel
    try:
        lr = LogisticRegression(max_iter=1000, random_state=42)
        selector = SelectFromModel(estimator=lr, max_features=num_features, prefit=False)
        selector.fit(X_train, y_train)
        selected_idx = selector.get_support(indices=True)
        return X_train[:, selected_idx], selected_idx
    except Exception as e:
        logging.error(f"Error in model-based (LR) feature selection: {e}")
        rfe = RFE(estimator=LogisticRegression(max_iter=1000, random_state=42), n_features_to_select=num_features)
        rfe.fit(X_train, y_train)
        selected_idx = rfe.get_support(indices=True)
        return X_train[:, selected_idx], selected_idx

def apply_extra_trees_importance(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on feature importances from an Extra Trees model.
    Falls back to mutual_info_classif based selection if an error occurs.
    """
    try:
        et = ExtraTreesClassifier(n_estimators=100, random_state=42)
        et.fit(X_train, y_train)
        importances = et.feature_importances_
        top_idx = np.argsort(importances)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in Extra Trees feature importance selection: {e}")
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector.get_support(indices=True)

def apply_embedded_elastic_net(X_train, y_train, num_features, **kwargs):
    """
    Embedded feature selection using Elastic Net.
    Falls back to LASSO if an error occurs.
    """
    try:
        en = ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42)
        en.fit(X_train, y_train)
        coefs = np.abs(en.coef_)
        top_idx = np.argsort(coefs)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in embedded Elastic Net feature selection: {e}")
        lasso = Lasso(alpha=0.01)
        lasso.fit(X_train, y_train)
        coefs = np.abs(lasso.coef_)
        top_idx = np.argsort(coefs)[-num_features:]
        return X_train[:, top_idx], top_idx

def apply_pca_loadings_selection(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on PCA loadings.
    Sums the absolute values of the PCA components and selects the top features.
    Falls back to chi2-based selection if any error occurs.
    """
    try:
        pca = PCA(n_components=num_features)
        pca.fit(X_train)
        loadings = np.abs(pca.components_).sum(axis=0)
        top_idx = np.argsort(loadings)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in PCA loadings feature selection: {e}")
        selector = SelectKBest(score_func=chi2, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector.get_support(indices=True)

def apply_pca_dictionary(X_train, y_train, num_features, **kwargs):
    """
    Extract features using PCA and return the transformed data as a 'dictionary' (placeholder).
    Falls back to TruncatedSVD if PCA fails.
    """
    try:
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca
    except Exception as e:
        logging.error(f"Error in PCA dictionary feature extraction: {e}")
        svd = TruncatedSVD(n_components=num_features, random_state=42)
        X_svd = svd.fit_transform(X_train)
        return X_svd, svd

def apply_vif_selection(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on Variance Inflation Factor (VIF).
    If an error occurs, falls back to f_classif based feature selection.
    """
    try:
        # Add a constant column for VIF calculation
        X_const = sm.add_constant(X_train)
        # Compute VIF for each feature, ignoring the constant (index 0)
        vif_scores = [variance_inflation_factor(X_const, i) for i in range(1, X_const.shape[1])]
        vif_array = np.array(vif_scores)
        # Select features with the smallest VIF (lowest multicollinearity)
        top_idx = np.argsort(vif_array)[:num_features]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in VIF feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector.get_support(indices=True)

def apply_stability_lasso_selection(X_train, y_train, num_features, **kwargs):
    """
    Stable feature selection using repeated LASSO runs.
    Averages the absolute coefficients over multiple runs and selects features with the highest average.
    Falls back to mutual_info_classif based selection on error.
    """
    try:
        n_runs = kwargs.get('n_runs', 10)
        alphas = kwargs.get('alphas', [0.01])
        coef_sum = np.zeros(X_train.shape[1])
        for i in range(n_runs):
            # Add a small noise to the data to stabilize the selection
            X_noisy = X_train + np.random.normal(0, 0.001, X_train.shape)
            model = Lasso(alpha=np.random.choice(alphas), random_state=42 + i)
            model.fit(X_noisy, y_train)
            coef_sum += np.abs(model.coef_)
        avg_coefs = coef_sum / n_runs
        top_idx = np.argsort(avg_coefs)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in stable LASSO feature selection: {e}")
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector.get_support(indices=True)

def apply_mutual_info_gain_ratio(X_train, y_train, num_features, **kwargs):
    """
    Feature selection using the ratio of mutual information to standard deviation.
    Falls back to mutual_info_classif based selection if an error occurs.
    """
    try:
        mi = mutual_info_classif(X_train, y_train)
        stds = np.std(X_train, axis=0)
        stds[stds == 0] = 1e-6  # Avoid division by zero
        gain_ratio = mi / stds
        top_idx = np.argsort(gain_ratio)[-num_features:]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in mutual information gain ratio feature selection: {e}")
        selector = SelectKBest(score_func=mutual_info_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector

def apply_chi2_pvalue_selection(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on the p-value from the Chi2 test.
    Falls back to chi2 based feature selection if an error occurs.
    """
    try:
        scores, pvalues = chi2(X_train, y_train)
        top_idx = np.argsort(pvalues)[:num_features]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in Chi2 p-value feature selection: {e}")
        selector = SelectKBest(score_func=chi2, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector

def apply_anova_pvalue_selection(X_train, y_train, num_features, **kwargs):
    """
    Feature selection based on p-values from the ANOVA test.
    Falls back to f_classif based selection if an error occurs.
    """
    try:
        scores, pvalues = f_classif(X_train, y_train)
        top_idx = np.argsort(pvalues)[:num_features]
        return X_train[:, top_idx], top_idx
    except Exception as e:
        logging.error(f"Error in ANOVA p-value feature selection: {e}")
        selector = SelectKBest(score_func=f_classif, k=num_features)
        X_sel = selector.fit_transform(X_train, y_train)
        return X_sel, selector

# ------------------- Feature Extraction Methods -------------------
def apply_pca(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Principal Component Analysis (PCA).
    Falls back to TruncatedSVD if PCA fails.
    """
    try:
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca
    except Exception as e:
        logging.error(f"Error in PCA feature extraction: {e}")
        svd = TruncatedSVD(n_components=num_features, random_state=42)
        X_svd = svd.fit_transform(X_train)
        return X_svd, svd

def apply_ica(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Independent Component Analysis (ICA).
    Falls back to PCA if ICA fails.
    """
    try:
        ica = FastICA(n_components=num_features, random_state=42)
        X_ica = ica.fit_transform(X_train)
        return X_ica, ica
    except Exception as e:
        logging.error(f"Error in ICA feature extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_t_sne(X_train, y_train, num_features, **kwargs):
    """
    Extract features using t-Distributed Stochastic Neighbor Embedding (t-SNE).
    Falls back to UMAP if t-SNE fails.
    """
    try:
        tsne = TSNE(n_components=num_features, random_state=42)
        X_tsne = tsne.fit_transform(X_train)
        return X_tsne, tsne
    except Exception as e:
        logging.error(f"Error in t-SNE feature extraction: {e}")
        from umap import UMAP
        um = UMAP(n_components=num_features, random_state=42)
        X_umap = um.fit_transform(X_train)
        return X_umap, um

def apply_umap_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using UMAP.
    Falls back to PCA if UMAP fails.
    """
    try:
        from umap import UMAP
        um = UMAP(n_components=num_features, random_state=42)
        X_umap = um.fit_transform(X_train)
        return X_umap, um
    except Exception as e:
        logging.error(f"Error in UMAP feature extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_kernel_pca_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Kernel PCA with the RBF kernel.
    Falls back to PCA if an error occurs.
    """
    try:
        kpca = KernelPCA(n_components=num_features, kernel='rbf', random_state=42)
        X_kpca = kpca.fit_transform(X_train)
        return X_kpca, kpca
    except Exception as e:
        logging.error(f"Error in Kernel PCA feature extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_truncated_svd_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Truncated SVD.
    Falls back to PCA if an error occurs.
    """
    try:
        svd = TruncatedSVD(n_components=num_features, random_state=42)
        X_svd = svd.fit_transform(X_train)
        return X_svd, svd
    except Exception as e:
        logging.error(f"Error in Truncated SVD feature extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_feature_agglomeration_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Feature Agglomeration.
    Falls back to PCA if it fails.
    """
    try:
        agg = FeatureAgglomeration(n_clusters=num_features)
        X_agg = agg.fit_transform(X_train)
        return X_agg, agg
    except Exception as e:
        logging.error(f"Error in Feature Agglomeration extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_isomap_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Isomap.
    Falls back to PCA if it fails.
    """
    try:
        iso = Isomap(n_components=num_features)
        X_iso = iso.fit_transform(X_train)
        return X_iso, iso
    except Exception as e:
        logging.error(f"Error in Isomap feature extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_gaussian_random_projection_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Gaussian Random Projection.
    Falls back to Sparse Random Projection if an error occurs.
    """
    from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
    try:
        grp = GaussianRandomProjection(n_components=num_features, random_state=42)
        X_grp = grp.fit_transform(X_train)
        return X_grp, grp
    except Exception as e:
        logging.error(f"Error in Gaussian Random Projection feature extraction: {e}")
        srp = SparseRandomProjection(n_components=num_features, random_state=42)
        X_srp = srp.fit_transform(X_train)
        return X_srp, srp

def apply_autoencoder_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using an Autoencoder (placeholder implementation).
    Falls back to PCA if autoencoder extraction is not implemented.
    """
    try:
        encoder = kwargs.get('encoder')
        if encoder is None:
            encoder = train_improved_autoencoder(X_train, encoding_dim=num_features, epochs=100, batch_size=32)
        X_auto = integrate_autoencoder_features(X_train, encoder, encoding_dim=num_features)
        return X_auto, encoder
    except Exception as e:
        logging.error(f"Error in Autoencoder feature extraction: {e}")
        pca = PCA(n_components=num_features)
        X_pca = pca.fit_transform(X_train)
        return X_pca, pca

def apply_truncated_pca(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Truncated PCA.
    Falls back to standard PCA if an error occurs.
    """
    try:
        svd_ = TruncatedSVD(n_components=num_features, random_state=42)
        X_svd = svd_.fit_transform(X_train)
        return X_svd, svd_
    except Exception as e:
        logging.error(f"Error in Truncated PCA feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_factor_analysis(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Factor Analysis.
    Falls back to PCA if an error occurs.
    """
    try:
        n_components_ = kwargs.get('n_components', 10)
        fa = FactorAnalysis(n_components=n_components_, random_state=42)
        X_fa = fa.fit_transform(X_train)
        return X_fa, fa
    except Exception as e:
        logging.error(f"Error in Factor Analysis feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_sparse_pca(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Sparse PCA.
    Falls back to FastICA if Sparse PCA fails.
    """
    from sklearn.decomposition import SparsePCA, FastICA
    try:
        n_components_ = kwargs.get('n_components', 10)
        alpha_ = kwargs.get('alpha', 1)
        spca = SparsePCA(n_components=n_components_, alpha=alpha_, random_state=42)
        X_spca = spca.fit_transform(X_train)
        return X_spca, spca
    except Exception as e:
        logging.error(f"Error in Sparse PCA feature extraction: {e}")
        ica = FastICA(n_components=num_features, random_state=42)
        X_ica = ica.fit_transform(X_train)
        return X_ica, ica

def apply_nmf(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Non-negative Matrix Factorization (NMF).
    Falls back to PCA if an error occurs.
    """
    from sklearn.decomposition import NMF
    try:
        n_components_ = kwargs.get('n_components', 10)
        init_ = kwargs.get('init', 'random')
        random_state_ = kwargs.get('random_state', 42)
        max_iter_ = kwargs.get('max_iter', 200)
        nmf = NMF(n_components=n_components_, init=init_, random_state=random_state_, max_iter=max_iter_)
        X_nmf = nmf.fit_transform(X_train)
        return X_nmf, nmf
    except Exception as e:
        logging.error(f"Error in NMF feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_fastica(X_train, y_train, num_features, **kwargs):
    """
    Extract features using FastICA.
    Falls back to PCA if an error occurs.
    """
    try:
        n_components_ = kwargs.get('n_components', 10)
        random_state_ = kwargs.get('random_state', 42)
        ica = FastICA(n_components=n_components_, random_state=random_state_)
        X_ica = ica.fit_transform(X_train)
        return X_ica, ica
    except Exception as e:
        logging.error(f"Error in FastICA feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_independent_component_analysis(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Independent Component Analysis (ICA).
    Falls back to PCA if ICA fails.
    """
    try:
        ica = FastICA(n_components=num_features, random_state=42)
        X_ica = ica.fit_transform(X_train)
        return X_ica, ica
    except Exception as e:
        logging.error(f"Error in Independent Component Analysis extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_locally_linear_embedding(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Locally Linear Embedding (LLE).
    Falls back to MDS if an error occurs.
    """
    try:
        n_components_ = kwargs.get('n_components', 10)
        lle = LocallyLinearEmbedding(n_components=n_components_, random_state=42)
        X_lle = lle.fit_transform(X_train)
        return X_lle, lle
    except Exception as e:
        logging.error(f"Error in Locally Linear Embedding extraction: {e}")
        mds_ = MDS(n_components=num_features, random_state=42)
        X_mds = mds_.fit_transform(X_train)
        return X_mds, mds_

def apply_mds(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Multidimensional Scaling (MDS).
    Falls back to Spectral Embedding if an error occurs.
    """
    try:
        n_components_ = kwargs.get('n_components', 10)
        random_state_ = kwargs.get('random_state', 42)
        mds_ = MDS(n_components=n_components_, random_state=random_state_)
        X_mds = mds_.fit_transform(X_train)
        return X_mds, mds_
    except Exception as e:
        logging.error(f"Error in MDS feature extraction: {e}")
        se_ = SpectralEmbedding(n_components=num_features, random_state=42)
        X_se = se_.fit_transform(X_train)
        return X_se, se_

def apply_isomap_alternative(X_train, y_train, num_features, **kwargs):
    """
    Alternative feature extraction using Isomap.
    Falls back to PCA if an error occurs.
    """
    try:
        iso = Isomap(n_components=num_features)
        X_iso = iso.fit_transform(X_train)
        return X_iso, iso
    except Exception as e:
        logging.error(f"Error in alternative Isomap feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_autoencoder_deep(X_train, y_train, num_features, **kwargs):
    """
    Extract features using a deep autoencoder (placeholder).
    If the autoencoder extraction fails, the original data is returned.
    """
    try:
        encoder = kwargs.get('encoder')
        if encoder is None:
            encoder = train_improved_autoencoder(X_train, encoding_dim=num_features, epochs=100, batch_size=32)
        X_auto = integrate_autoencoder_features(X_train, encoder, encoding_dim=num_features)
        return X_auto, encoder
    except Exception as e:
        logging.error(f"Error in deep autoencoder feature extraction: {e}")
        encoder = None
        return X_train, encoder

def apply_sparse_random_projection_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Sparse Random Projection.
    Falls back to Gaussian Random Projection if an error occurs.
    """
    from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection
    try:
        srp = SparseRandomProjection(n_components=num_features, random_state=42)
        X_srp = srp.fit_transform(X_train)
        return X_srp, srp
    except Exception as e:
        logging.error(f"Error in Sparse Random Projection extraction: {e}")
        grp = GaussianRandomProjection(n_components=num_features, random_state=42)
        X_grp = grp.fit_transform(X_train)
        return X_grp, grp

def apply_lda_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Linear Discriminant Analysis (LDA).
    Falls back to PCA if LDA fails.
    """
    try:
        lda = LinearDiscriminantAnalysis(n_components=num_features)
        X_lda = lda.fit_transform(X_train, y_train)
        return X_lda, lda
    except Exception as e:
        logging.error(f"Error in LDA feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_spectral_embedding_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Spectral Embedding.
    Falls back to PCA if an error occurs.
    """
    try:
        se_ = SpectralEmbedding(n_components=num_features, random_state=42)
        X_se = se_.fit_transform(X_train)
        return X_se, se_
    except Exception as e:
        logging.error(f"Error in Spectral Embedding extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_mds_different_params(X_train, y_train, num_features, **kwargs):
    """
    Extract features using MDS with different parameters (a variation).
    Falls back to UMAP if an error occurs.
    """
    try:
        mds_ = MDS(n_components=num_features, random_state=42)
        X_mds = mds_.fit_transform(X_train)
        return X_mds, mds_
    except Exception as e:
        logging.error(f"Error in MDS with different parameters: {e}")
        from umap import UMAP
        um_ = UMAP(n_components=num_features, random_state=42)
        X_umap = um_.fit_transform(X_train)
        return X_umap, um_

def apply_t_sne_enhanced(X_train, y_train, num_features, **kwargs):
    """
    Extract features using an enhanced version of t-SNE.
    Falls back to UMAP if enhanced t-SNE fails.
    """
    try:
        tsne = TSNE(n_components=num_features, random_state=42)
        X_tsne = tsne.fit_transform(X_train)
        return X_tsne, tsne
    except Exception as e:
        logging.error(f"Error in enhanced t-SNE feature extraction: {e}")
        from umap import UMAP
        umap_ = UMAP(n_components=num_features, random_state=42)
        X_umap = umap_.fit_transform(X_train)
        return X_umap, umap_

def apply_feature_hashing(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Feature Hashing.
    Falls back to PCA if an error occurs.
    """
    from sklearn.feature_extraction import FeatureHasher
    try:
        hasher = FeatureHasher(n_features=num_features, input_type="matrix")
        X_hashed = hasher.transform(X_train)
        X_hashed = X_hashed.toarray()  # Convert sparse matrix to dense array
        return X_hashed, hasher
    except Exception as e:
        logging.error(f"Error in Feature Hashing extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_umap_min_dist_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using UMAP with a specified minimum distance (min_dist).
    Falls back to PCA if an error occurs.
    """
    try:
        from umap import UMAP
        min_dist_ = kwargs.get('min_dist', 0.1)
        umap_ = UMAP(n_components=num_features, random_state=42, min_dist=min_dist_)
        X_umap = umap_.fit_transform(X_train)
        return X_umap, umap_
    except Exception as e:
        logging.error(f"Error in UMAP (min_dist) feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_kernel_pca_poly_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Kernel PCA with a polynomial kernel.
    Falls back to PCA if an error occurs.
    """
    try:
        degree_ = kwargs.get('degree', 3)
        kpca_ = KernelPCA(n_components=num_features, kernel='poly', degree=degree_, random_state=42)
        X_kpca = kpca_.fit_transform(X_train)
        return X_kpca, kpca_
    except Exception as e:
        logging.error(f"Error in Kernel PCA (poly) feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

def apply_diffusion_map_extraction(X_train, y_train, num_features, **kwargs):
    """
    Extract features using Diffusion Map via Spectral Embedding with nearest neighbor affinity.
    Falls back to PCA if an error occurs.
    """
    from sklearn.manifold import SpectralEmbedding
    try:
        diffusion = SpectralEmbedding(n_components=num_features, affinity='nearest_neighbors', random_state=42)
        X_diff = diffusion.fit_transform(X_train)
        return X_diff, diffusion
    except Exception as e:
        logging.error(f"Error in Diffusion Map feature extraction: {e}")
        pca_ = PCA(n_components=num_features)
        X_pca_ = pca_.fit_transform(X_train)
        return X_pca_, pca_

# =============================================================================
# Mapping dictionaries for Feature Selection Functions and Classifiers
# =============================================================================

# Dictionary mapping names to corresponding feature selection function objects.
feature_selector_functions = {
    "apply_chi_square": apply_chi_square,
    "apply_correlation_coefficient": apply_correlation_coefficient,
    "apply_mutual_information": apply_mutual_information,
    "apply_variance_threshold": apply_variance_threshold,
    "apply_anova_f_test": apply_anova_f_test,
    "apply_information_gain": apply_information_gain,
    "apply_univariate_feature_selection": apply_univariate_feature_selection,
    "apply_lasso_fs": apply_lasso_fs,
    "apply_rfe_fs": apply_rfe_fs,
    "apply_elastic_net_fs": apply_elastic_net_fs,
    "apply_sequential_feature_selector_func": apply_sequential_feature_selector_func,
    "apply_select_fdr": apply_select_fdr,
    "apply_select_fwe": apply_select_fwe,
    "apply_feature_importance_rf": apply_feature_importance_rf,
    "apply_permutation_importance_fs": apply_permutation_importance_fs,
    "apply_relief_f": apply_relief_f,
    "apply_mutual_info_selection": apply_mutual_info_selection,
    "apply_select_from_model_lr": apply_select_from_model_lr,
    "apply_extra_trees_importance": apply_extra_trees_importance,
    "apply_embedded_elastic_net": apply_embedded_elastic_net,
    "apply_pca_loadings_selection": apply_pca_loadings_selection,
    "apply_pca_dictionary": apply_pca_dictionary,
    "apply_vif_selection": apply_vif_selection,
    "apply_stability_lasso_selection": apply_stability_lasso_selection,
    "apply_mutual_info_gain_ratio": apply_mutual_info_gain_ratio,
    "apply_chi2_pvalue_selection": apply_chi2_pvalue_selection,
    "apply_anova_pvalue_selection": apply_anova_pvalue_selection,
    # Feature extraction functions:
    "apply_pca": apply_pca,
    "apply_ica": apply_ica,
    "apply_t_sne": apply_t_sne,
    "apply_umap_extraction": apply_umap_extraction,
    "apply_kernel_pca_extraction": apply_kernel_pca_extraction,
    "apply_truncated_svd_extraction": apply_truncated_svd_extraction,
    "apply_feature_agglomeration_extraction": apply_feature_agglomeration_extraction,
    "apply_isomap_extraction": apply_isomap_extraction,
    "apply_gaussian_random_projection_extraction": apply_gaussian_random_projection_extraction,
    "apply_autoencoder_extraction": apply_autoencoder_extraction,
    "apply_truncated_pca": apply_truncated_pca,
    "apply_factor_analysis": apply_factor_analysis,
    "apply_sparse_pca": apply_sparse_pca,
    "apply_nmf": apply_nmf,
    "apply_fastica": apply_fastica,
    "apply_independent_component_analysis": apply_independent_component_analysis,
    "apply_locally_linear_embedding": apply_locally_linear_embedding,
    "apply_mds": apply_mds,
    "apply_isomap_alternative": apply_isomap_alternative,
    "apply_autoencoder_deep": apply_autoencoder_deep,
    "apply_sparse_random_projection_extraction": apply_sparse_random_projection_extraction,
    "apply_lda_extraction": apply_lda_extraction,
    "apply_spectral_embedding_extraction": apply_spectral_embedding_extraction,
    "apply_mds_different_params": apply_mds_different_params,
    "apply_t_sne_enhanced": apply_t_sne_enhanced,
    "apply_feature_hashing": apply_feature_hashing,
    "apply_umap_min_dist_extraction": apply_umap_min_dist_extraction,
    "apply_kernel_pca_poly_extraction": apply_kernel_pca_poly_extraction,
    "apply_diffusion_map_extraction": apply_diffusion_map_extraction,
}

# =============================================================================
# Classifier Dictionaries
# =============================================================================
INVOLVED_CLASSIFIERS = {
    'Decision_Tree': {
        'model': DecisionTreeClassifier,
        'params': {'random_state': 42, 'class_weight': 'balanced'},
        'param_grid': {'max_depth': [None, 10, 20, 30],
                       'min_samples_split': [2, 5, 10],
                       'min_samples_leaf': [1, 2, 4],
                       'criterion': ['gini', 'entropy']}
    },
    'Logistic_Regression': {
        'model': LogisticRegression,
        'params': {'max_iter': 1000, 'random_state': 42, 'class_weight': 'balanced'},
        'param_grid': {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                       'penalty': ['l2'],
                       'solver': ['lbfgs', 'saga']}
    },
    'Linear_Discriminant_Analysis': {
        'model': LinearDiscriminantAnalysis,
        'params': {},
        'param_grid': [{'solver': ['svd']},
                       {'solver': ['lsqr', 'eigen'], 'shrinkage': [None, 'auto']}]
    },
    'Gaussian_Naive_Bayes': {
        'model': GaussianNB,
        'params': {},
        'param_grid': {'var_smoothing': [1e-09, 1e-08, 1e-07]}
    },
    'Bernoulli_Naive_Bayes': {
        'model': BernoulliNB,
        'params': {'alpha': 1.0, 'binarize': 0.0},
        'param_grid': {'alpha': [0.1, 1.0, 10.0],
                       'binarize': [0.0, 0.5, 1.0]}
    },
    'Complement_Naive_Bayes': {
        'model': ComplementNB,
        'params': {'alpha': 1.0},
        'param_grid': {'alpha': [0.1, 1.0, 10.0]}
    },
    'K_Nearest_Neighbors': {
        'model': KNeighborsClassifier,
        'params': {'n_neighbors': 5},
        'param_grid': {'n_neighbors': [3, 5, 7, 9],
                       'weights': ['uniform', 'distance'],
                       'metric': ['euclidean', 'manhattan', 'minkowski']}
    },
    'Random_Forest': {
        'model': RandomForestClassifier,
        'params': {'random_state': 42, 'class_weight': 'balanced'},
        'param_grid': {'n_estimators': [50, 100, 200, 400],
                       'max_depth': [None, 10, 20, 30, 50],
                       'min_samples_split': [2, 5, 10, 15],
                       'min_samples_leaf': [1, 2, 4, 8]}
    },
    'Ridge_Classifier': {
        'model': RidgeClassifier,
        'params': {'alpha': 1.0, 'random_state': 42},
        'param_grid': {'alpha': [0.1, 1.0, 10.0, 100.0]}
    },
    'Support_Vector_Machine': {
        'model': SVC,
        'params': {'random_state': 42, 'class_weight': 'balanced'},
        'param_grid': {'C': [0.01, 0.1, 1, 10],
                       'kernel': ['linear', 'rbf', 'poly'],
                       'gamma': ['scale', 'auto']}
    },
    'XGBoost': {
        'model': XGBClassifier,
        'params': {'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss'},
        'param_grid': {'n_estimators': [50, 100, 200, 300],
                       'learning_rate': [0.001, 0.01, 0.1, 0.2],
                       'max_depth': [3, 5, 7, 9]}
    },
    'LightGBM': {
        'model': LGBMClassifier,
        'params': {'random_state': 42},
        'param_grid': {'n_estimators': [50, 100, 200],
                       'learning_rate': [0.01, 0.1, 0.2],
                       'max_depth': [-1, 10, 20]}
    },
    'MLP_Classifier': {
        'model': lambda: MLPClassifier(random_state=42),
        'params': {'random_state': 42},
        'param_grid': {
            'hidden_layer_sizes': [(100,), (50, 50), (100, 50, 25)],
            'activation': ['relu', 'tanh', 'logistic'],
            'solver': ['adam', 'sgd'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive']
        }
    },
    'Stacking': {
        'model': lambda: StackingClassifier(
            estimators=[("dt", DecisionTreeClassifier(random_state=42)),
                        ("knn", KNeighborsClassifier(n_neighbors=5))],
            final_estimator=LogisticRegression(random_state=42, max_iter=1000)
        ),
        'params': {},
        'param_grid': {}
    },
    'Voting_Classifier': {
        'model': lambda: VotingClassifier(
            estimators=[("lr", LogisticRegression(max_iter=1000, random_state=42)),
                        ("dt", DecisionTreeClassifier(random_state=42))],
            voting='soft', n_jobs=-1
        ),
        'params': {},
        'param_grid': {}
    },
    'ExtraTrees': {
        'model': ExtraTreesClassifier,
        'params': {'random_state': 42, 'n_estimators': 100},
        'param_grid': {'n_estimators': [50, 100, 200],
                       'max_depth': [None, 10, 20]}
    },
    'AdaBoost': {
        'model': AdaBoostClassifier,
        'params': {'random_state': 42, 'n_estimators': 50},
        'param_grid': {'n_estimators': [50, 100, 200],
                       'learning_rate': [0.01, 0.1, 1]}
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier,
        'params': {'random_state': 42},
        'param_grid': {'n_estimators': [50, 100, 200],
                       'learning_rate': [0.01, 0.1, 0.2],
                       'max_depth': [3, 5, 7]}
    },
    'Bagging_Classifier': {
        'model': BaggingClassifier,
        'params': {'random_state': 42},
        'param_grid': {'n_estimators': [10, 50, 100],
                       'max_samples': [0.5, 1.0],
                       'max_features': [0.5, 1.0],
                       'bootstrap': [True, False]}
    },
    'HistGradientBoosting': {
        'model': HistGradientBoostingClassifier,
        'params': {'random_state': 42},
        'param_grid': {'max_iter': [100, 200],
                       'learning_rate': [0.01, 0.1],
                       'max_depth': [3, 5, 10],
                       'min_samples_leaf': [20, 50]}
    },
    'NearestCentroid': {
        'model': NearestCentroid,
        'params': {'metric': 'euclidean'},
        'param_grid': {}
    },
    'ExtraTreesClassifier_Variant': {
        'model': ExtraTreesClassifier,
        'params': {'random_state': 42},
        'param_grid': {'n_estimators': [50, 100, 200],
                       'max_depth': [None, 10, 20],
                       'min_samples_split': [2, 5],
                       'min_samples_leaf': [1, 2]}
    },
    'Dummy_Classifier': {
        'model': DummyClassifier,
        'params': {},
        'param_grid': {'strategy': ['most_frequent', 'stratified', 'uniform', 'constant'],
                       'constant': [0, 1]}
    },
    'GaussianProcess': {
        'model': GaussianProcessClassifier,
        'params': {'random_state': 42},
        'param_grid': {'max_iter_predict': [100, 200],
                       'multi_class': ['one_vs_rest', 'one_vs_one']}
    },
    'ExtraTreesClassifier_New': {
        'model': ExtraTreesClassifier,
        'params': {'random_state': 42, 'n_estimators': 150},
        'param_grid': {'n_estimators': [100, 150, 200],
                       'max_depth': [None, 10, 20]}
    },
    'GaussianNB_Variant': {
        'model': GaussianNB,
        'params': {'priors': None},
        'param_grid': {}
    },
    'DecisionStump': {
        'model': DecisionTreeClassifier,
        'params': {'max_depth': 1, 'random_state': 42},
        'param_grid': {}
    },
    'LogisticRegressionCV': {
        'model': LogisticRegression,
        'params': {'max_iter': 1000, 'cv': 5, 'random_state': 42, 'class_weight': 'balanced'},
        'param_grid': {}
    },
    'SGDClassifier_Variant': {
        'model': SGDClassifier,
        'params': {'loss': 'hinge', 'max_iter': 1000, 'random_state': 42},
        'param_grid': {'alpha': [0.0001, 0.001, 0.01],
                       'penalty': ['l2', 'l1', 'elasticnet']}
    }
}

# Define the feature selectors mapping using a dictionary.
INVOLVED_FEATURE_SELECTORS = {
    "fs-Chi_Square_Test": {"function": "apply_chi_square", "params": {}},
    "fs-Correlation_Coefficient": {"function": "apply_correlation_coefficient", "params": {}},
    "fs-Mutual_Information": {"function": "apply_mutual_information", "params": {}},
    "fs-Variance_Threshold": {"function": "apply_variance_threshold", "params": {}},
    "fs-ANOVA_F_Test": {"function": "apply_anova_f_test", "params": {}},
    "fs-Information_Gain": {"function": "apply_information_gain", "params": {}},
    "fs-Univariate_Feature_Selection": {"function": "apply_univariate_feature_selection", "params": {"score_func": mutual_info_classif}},
    "fs-LASSO_FS": {"function": "apply_lasso_fs", "params": {"alpha": 0.01}},
    "fs-RFE": {"function": "apply_rfe_fs", "params": {}},
    "fs-ElasticNet_FS": {"function": "apply_elastic_net_fs", "params": {"alpha": 0.01, "l1_ratio": 0.5}},
    "fs-Sequential_FS": {"function": "apply_sequential_feature_selector_func", "params": {"direction": "forward", "scoring": "accuracy", "cv": 3}},
    "fs-FDR_FS": {"function": "apply_select_fdr", "params": {"score_func": f_classif, "alpha": 0.05}},
    "fs-FWE_FS": {"function": "apply_select_fwe", "params": {"score_func": f_classif, "alpha": 0.05}},
    "fs-Feature_Importance_RF": {"function": "apply_feature_importance_rf", "params": {}},
    "fs-Permutation_Importance": {"function": "apply_permutation_importance_fs", "params": {"scoring": "accuracy"}},
    "fs-ReliefF": {"function": "apply_relief_f", "params": {"n_features_to_select": 10}},
    "fs-Mutual_Info_Selection": {"function": "apply_mutual_info_selection", "params": {}},
    "fs-Select_From_Model_LR": {"function": "apply_select_from_model_lr", "params": {}},
    "fs-Extra_Trees_Importance": {"function": "apply_extra_trees_importance", "params": {}},
    "fs-Embedded_ElasticNet": {"function": "apply_embedded_elastic_net", "params": {}},
    "fs-VIF_Selection": {"function": "apply_vif_selection", "params": {}},
    "fs-Stability_Lasso": {"function": "apply_stability_lasso_selection", "params": {"n_runs": 10, "alphas": [0.01]}},
    "fs-Mutual_Info_Gain_Ratio": {"function": "apply_mutual_info_gain_ratio", "params": {}},
    "fs-Chi2_Pvalue": {"function": "apply_chi2_pvalue_selection", "params": {}},
    "fs-ANOVA_Pvalue": {"function": "apply_anova_pvalue_selection", "params": {}},
    # Feature Extraction selectors
    "fs-PCA_Loadings_Selection": {"function": "apply_pca_loadings_selection", "params": {}},
    "fs-PCA_Dictionary": {"function": "apply_pca_dictionary", "params": {}},
    "FE-PCA": {"function": "apply_pca", "params": {}},
    "FE-ICA": {"function": "apply_ica", "params": {}},
    "FE-t_SNE": {"function": "apply_t_sne", "params": {}},
    "FE-UMAP": {"function": "apply_umap_extraction", "params": {}},
    "FE-Kernel_PCA": {"function": "apply_kernel_pca_extraction", "params": {}},
    "FE-Truncated_SVD": {"function": "apply_truncated_svd_extraction", "params": {}},
    "FE-Feature_Agglomeration": {"function": "apply_feature_agglomeration_extraction", "params": {}},
    "FE-Isomap": {"function": "apply_isomap_extraction", "params": {}},
    "FE-Gaussian_Random_Projection": {"function": "apply_gaussian_random_projection_extraction", "params": {}},
    "FE-Autoencoder": {"function": "apply_autoencoder_extraction", "params": {}},
    "FE-Truncated_PCA": {"function": "apply_truncated_pca", "params": {}},
    "FE-Factor_Analysis": {"function": "apply_factor_analysis", "params": {"n_components": 10}},
    "FE-Sparse_PCA": {"function": "apply_sparse_pca", "params": {"n_components": 10, "alpha": 1}},
    "FE-NMF": {"function": "apply_nmf", "params": {"n_components": 10, "init": "random", "random_state": RANDOM_SEED, "max_iter": 500}},
    "FE-FastICA": {"function": "apply_fastica", "params": {"n_components": 10, "random_state": RANDOM_SEED}},
    "FE-Independent_Component_Analysis": {"function": "apply_independent_component_analysis", "params": {}},
    "FE-Locally_Linear_Embedding": {"function": "apply_locally_linear_embedding", "params": {"n_components": 10}},
    "FE-MDS": {"function": "apply_mds", "params": {"n_components": 10, "random_state": RANDOM_SEED}},
    "FE-Isomap_Alternative": {"function": "apply_isomap_alternative", "params": {}},
    "FE-Autoencoder_Deep": {"function": "apply_autoencoder_deep", "params": {}},
    "FE-Sparse_Random_Projection": {"function": "apply_sparse_random_projection_extraction", "params": {}},
    "FE-LDA_Extraction": {"function": "apply_lda_extraction", "params": {}},
    "FE-Spectral_Embedding": {"function": "apply_spectral_embedding_extraction", "params": {}},
    "FE-MDS_Different_Params": {"function": "apply_mds_different_params", "params": {}},
    "FE-t_SNE_Enhanced": {"function": "apply_t_sne_enhanced", "params": {}},
    "FE-Feature_Hashing": {"function": "apply_feature_hashing", "params": {}},
    "FE-UMAP_Min_Dist": {"function": "apply_umap_min_dist_extraction", "params": {"min_dist": 0.1}},
    "FE-Kernel_PCA_Poly": {"function": "apply_kernel_pca_poly_extraction", "params": {"degree": 3}},
    "FE-Diffusion_Map": {"function": "apply_diffusion_map_extraction", "params": {}},
}

# =============================================================================
# Start Processing Each Dataset
# =============================================================================
for dataset_file in dataset_files:
    try:
        # Print the current dataset processing status
        print(f"\n******** Processing Dataset: {dataset_file} ********")
        # Build the full file path to the dataset file
        FILE_PATH = dataset_file
        # Extract dataset name by removing the file extension
        dataset_name = os.path.splitext(os.path.basename(dataset_file))[0]
        # Create a timestamp string for the results filename
        timestamp = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        # Define the results Excel file path (inside a "res" folder)
        # Determine output folder: if user didn't set RESULTS_FOLDER, use default "res"
        output_dir = RESULTS_FOLDER if RESULTS_FOLDER else os.path.join(os.getcwd(), 'res')
        results_excel_path = os.path.join(output_dir, f"results_{dataset_name}_{timestamp}.xlsx")
        # Make sure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        # Create the directory for results if it does not exist
        if not os.path.exists(os.path.dirname(results_excel_path)):
            os.makedirs(os.path.dirname(results_excel_path))

        # Create an empty initial Excel file to store results
        pd.DataFrame().to_excel(results_excel_path, index=False)

        # Read data from file, supporting both Excel and CSV formats
        if FILE_PATH.endswith(('.xlsx', '.xls')):
            Org_Data = pd.read_excel(FILE_PATH, sheet_name="Sheet1")
        elif FILE_PATH.endswith('.csv'):
            Org_Data = pd.read_csv(FILE_PATH)
        else:
            print(f"File format of {dataset_file} is not supported. Skipping further processing.")
            continue
        cols = Org_Data.columns.tolist()
        cols[0] = 'PatientID'
        cols[-1] = 'Outcome'
        Org_Data.columns = cols

        # Assume that the outcome column is the last column in the dataset
        outcome_col = Org_Data.columns[-1]
        # Use the predefined ANALYSIS_MODE from the configuration
        if ANALYSIS_MODE == "Semi-Supervised":
            # In semi-supervised mode, fill missing Outcome values with the mode (most frequent value)
            try:
                mode_value = Org_Data[outcome_col].mode()[0]
                Org_Data[outcome_col].fillna(mode_value, inplace=True)
            except Exception as e:
                logging.error(f"Error filling missing Outcome values: {e}")
        elif ANALYSIS_MODE == "Supervised":
            # In supervised mode, drop rows where Outcome is missing
            Org_Data = Org_Data.dropna(subset=[outcome_col])
        else:
            raise ValueError("ANALYSIS_MODE must be 'Supervised' or 'Semi-Supervised'")

        # ----------------------------------------
        # Filter and sample class-wise data based on selection percent
        # ----------------------------------------
        filtered_data = pd.DataFrame()

        for class_name, frac in CLASS_SELECTION_PERCENT.items():
            class_subset = Org_Data[Org_Data[outcome_col] == class_name]
            selected = class_subset.sample(frac=frac, random_state=RANDOM_SEED)
            filtered_data = pd.concat([filtered_data, selected], axis=0)

        filtered_data = filtered_data.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

        # نمایش توزیع کلاس‌ها قبل و بعد از فیلتر کردن
        show_class_distribution(Org_Data, filtered_data, outcome_col=outcome_col)

        # Assume the first column is Patient_ID, the last column is Outcome, and the rest are features
        Patient_ID = filtered_data['PatientID']
        Data = filtered_data.iloc[:, 1:-1]
        Outcome = filtered_data.iloc[:, -1].map(CLASS_MAPPING)
        # Raise an error if any Outcome class could not be mapped
        if Outcome.isnull().any():
            raise ValueError("Some Outcome classes do not have a mapping.")

        # Select only numeric columns from the features
        numeric_columns = Data.select_dtypes(include=[np.number]).columns
        numeric_data = Data[numeric_columns]

        # ---------------------------------------------------------------
        # Train/Test split – based on TEST_MODE
        # ---------------------------------------------------------------
        if TEST_MODE == 'External':
            if not os.path.exists(EXTERNAL_TEST_FILE):
                raise FileNotFoundError(f"External test file '{EXTERNAL_TEST_FILE}' not found.")

            ext_df = load_dataframe(EXTERNAL_TEST_FILE)
            cols = ext_df.columns.tolist()
            cols[0] = 'PatientID'
            cols[-1] = 'Outcome'
            ext_df.columns = cols

            X_test = ext_df.reindex(columns=numeric_columns, fill_value=np.nan)
            y_test = ext_df['Outcome'].map(CLASS_MAPPING)
            Patient_ID_test = ext_df['PatientID']

            X_train = numeric_data
            y_train = Outcome
            Patient_ID_train = Patient_ID

        else:  # Internal test mode
            X_train, X_test, y_train, y_test, Patient_ID_train, Patient_ID_test = train_test_split(
                numeric_data, Outcome, Patient_ID,
                test_size=TEST_SIZE, stratify=Outcome, random_state=RANDOM_SEED
            )

        # Imputation
        imputer = SimpleImputer(strategy=IMPUTATION_STRATEGY)
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed  = imputer.transform(X_test)

        if SCALING_METHOD == 'MinMaxScaler':
            scaler = MinMaxScaler()
        elif SCALING_METHOD == 'StandardScaler':
            scaler = StandardScaler()
        elif SCALING_METHOD == 'RobustScaler':
            scaler = RobustScaler()
        elif SCALING_METHOD == 'Normalizer':
            scaler = Normalizer()
        elif SCALING_METHOD == 'MaxAbsScaler':
            scaler = MaxAbsScaler()
        else:
            scaler = None

        if scaler is not None:
            X_train_scaled = scaler.fit_transform(X_train_imputed)
            X_test_scaled  = scaler.transform(X_test_imputed)
        else:
            X_train_scaled = X_train_imputed
            X_test_scaled  = X_test_imputed


        # Save a summary of the preprocessing into the results file in a "Preprocessing" sheet
        preproc_df = pd.DataFrame({
            "Num Samples": [len(Outcome)],
            "Unique Outcomes": [', '.join(map(str, Outcome.unique()))],
            "Analysis Mode": [ANALYSIS_MODE]
        })
        preproc_df.to_excel(results_excel_path, sheet_name="Preprocessing", index=False)

        # Create a StratifiedKFold for cross-validation
        skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

        # Lists to store selected features and best parameters for later reporting
        selected_features_all = []
        best_parameters_all = []
        # Loop over Outcome classifiers (only those selected in the OUTCOME_CLASSIFIERS list)
        # Skip training Outcome Classifier if in 'Supervised' mode
       # Outcome Classifier Logic: Only used in Semi-Supervised mode
        # === Outcome Classifier Phase ===
        if ANALYSIS_MODE == "Supervised":
            logging.info("=== ANALYSIS_MODE is 'Supervised': Outcome classifier skipped. ===")

            # 🧼 پاکسازی داده‌های تست که outcome ندارن
            test_mask = ~y_test.isnull()
            X_test = X_test[test_mask]
            X_test_scaled = X_test_scaled[test_mask]
            y_test = y_test[test_mask]
            Patient_ID_test = Patient_ID_test[test_mask]

            # outcomeها همون داده‌های واقعی هستن، نه پیش‌بینی شده
            y_train_pred = y_train.copy()
            y_test_pred = y_test.copy()

            outcome_clf_names = ["Supervised_Mode"]

        elif ANALYSIS_MODE == "Semi-Supervised":
            outcome_clf_names = OUTCOME_CLASSIFIERS
            for outcome_clf_name in outcome_clf_names:
                print(f"\n=== Outcome Classifier: {outcome_clf_name} ===")
                oc_info = INVOLVED_CLASSIFIERS.get(outcome_clf_name)
                if oc_info is None:
                    print(f"Classifier {outcome_clf_name} not found.")
                    continue

                try:
                    oc_model = oc_info['model'](**oc_info['params'])
                    oc_model.fit(X_train_scaled, y_train)

                    ext_df = load_dataframe(EXTERNAL_TEST_FILE)
                    ext_feats = ext_df.reindex(columns=numeric_columns, fill_value=np.nan)
                    ext_imp = imputer.transform(ext_feats)
                    ext_scaled = scaler.transform(ext_imp) if scaler is not None else ext_imp

                    y_orig = ext_df['Outcome'].map(CLASS_MAPPING)
                    missing_mask = y_orig.isna()
                    y_pred = oc_model.predict(ext_scaled[missing_mask])
                    y_test = y_orig.copy()
                    y_test.loc[missing_mask] = y_pred

                    X_test = ext_feats
                    X_test_scaled = ext_scaled
                    Patient_ID_test = ext_df['PatientID']

                    y_train_pred = y_train.copy()
                    y_test_pred = y_test.copy()

                except Exception as e:
                    print(f"Error training Outcome Classifier {outcome_clf_name}: {e}")
                    continue
        else:
            raise ValueError("ANALYSIS_MODE must be either 'Supervised' or 'Semi-Supervised'")


        for outcome_clf_name in outcome_clf_names:


            # Record best parameters for each feature selector and analysis classifier
            best_params_record = {}
            # Loop through each feature selector
            for fs_name in INVOLVED_FEATURE_SELECTORS.keys():
                fs_info = INVOLVED_FEATURE_SELECTORS[fs_name]
                print(f"\n--- Running Feature Selector: {fs_name} for Outcome {outcome_clf_name} ---")
                fs_func_name = fs_info['function']
                fs_params = fs_info['params']
                # Fetch the corresponding function object from the mapping dictionary
                fs_function = feature_selector_functions.get(fs_func_name)
                if fs_function is None:
                    print(f"Function '{fs_func_name}' for {fs_name} not found.")
                    continue

                try:
                    # Use the predicted Outcome as the label for feature selection
                    X_train_selected, selector_obj = fs_function(X_train_scaled, y_train_pred, NOF, **fs_params)
                except Exception as e:
                    print(f"Error in Feature Selector {fs_name}: {e}.")
                    continue

                # Extract the names of the selected features
                sel_feature_names = []
                if selector_obj is not None:
                    if hasattr(selector_obj, 'get_support'):
                        sel_feature_names = numeric_columns[selector_obj.get_support()].tolist()
                    elif isinstance(selector_obj, (np.ndarray, list)):
                        sel_feature_names = numeric_columns.take(selector_obj).tolist()
                    else:
                        sel_feature_names = numeric_columns.tolist()
                else:
                    sel_feature_names = numeric_columns.tolist()

                sel_feature_names = [str(f) for f in sel_feature_names]
                selected_features_all.append({
                    'Outcome Classifier': outcome_clf_name,
                    'Feature Selector': fs_name,
                    'Selected Features': ', '.join(sel_feature_names)
                })
                append_df_to_excel(
                    results_excel_path,
                    pd.DataFrame([{
                        'Outcome Classifier': outcome_clf_name,
                        'Feature Selector': fs_name,
                        'Selected Features': ', '.join(sel_feature_names)
                    }]),
                    sheet_name="Selected_Features"
                )

                # Extract the selected features from the test set using the same selector object
                if selector_obj is not None:
                    if hasattr(selector_obj, 'transform'):
                        try:
                            X_test_selected = selector_obj.transform(X_test_scaled)
                        except:
                            X_test_selected = X_test_scaled
                    elif isinstance(selector_obj, (np.ndarray, list)):
                        X_test_selected = X_test_scaled[:, selector_obj]
                    else:
                        X_test_selected = X_test_scaled
                else:
                    X_test_selected = X_test_scaled

                # Loop over each analysis classifier defined in the INVOLVED_CLASSIFIERS dictionary
                for clf_name in INVOLVED_CLASSIFIERS.keys():
                    print(f"\n--- Grid Search for Analysis Classifier {clf_name} with Feature Selector {fs_name} for Outcome {outcome_clf_name} ---")
                    print(f"Classifier_Outcom: {outcome_clf_name}  |  Classifier_Analysis: {clf_name}")
                    clf_info = INVOLVED_CLASSIFIERS[clf_name]
                    model_class = clf_info['model']
                    model_initial_params = clf_info['params'].copy()
                    param_grid = clf_info.get('param_grid', {})

                    # Build the initial classifier model
                    try:
                        if callable(model_class):
                            model = model_class()
                            for k, v in model_initial_params.items():
                                setattr(model, k, v)
                        else:
                            model = model_class(**model_initial_params)
                    except Exception as e:
                        print(f"Error initializing classifier {clf_name}: {e}.")
                        continue

                    # Use RandomizedSearchCV to find the best hyperparameters
                    try:
                        search = RandomizedSearchCV(
                            estimator=model,
                            param_distributions=param_grid,
                            n_iter=5,
                            cv=3,
                            scoring='accuracy',
                            random_state=RANDOM_SEED,
                            n_jobs=-1,
                            verbose=1
                        )
                        search.fit(X_train_selected, y_train_pred)
                        best_params = search.best_params_
                        best_score = search.best_score_
                        print(f"Best parameters for {clf_name}: {best_params} (CV Accuracy: {best_score:.4f})")
                    except Exception as e:
                        print(f"Grid search for {clf_name} encountered an error: {e}. Using initial parameters.")
                        best_params = model_initial_params

                    # Record the best parameters found during hyperparameter search
                    best_params_record.setdefault(fs_name, {})[clf_name] = best_params
                    best_params_df = pd.DataFrame([{
                        'Outcome Classifier': outcome_clf_name,
                        'Feature Selector': fs_name,
                        'Classifier': clf_name,
                        **best_params
                    }])
                    append_df_to_excel(results_excel_path, best_params_df, sheet_name="Best_Parameters")

                    # Prepare for cross-validation of the current analysis classifier with the selected features
                    fold_results = []
                    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_selected, y_train), start=1):

                        try:
                            if callable(model_class):
                                model_fold = model_class()
                                for k, v in {**model_initial_params, **best_params}.items():
                                    setattr(model_fold, k, v)
                            else:
                                model_fold = model_class(**{**model_initial_params, **best_params})
                        except Exception as e:
                            print(f"Error initializing model {clf_name} in Fold {fold}: {e}.")
                            continue

                        try:
                            X_train_fold, X_val_fold = X_train_selected[train_idx], X_train_selected[val_idx]
                            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
                        except Exception as e:
                            print(f"Error splitting data in Fold {fold}: {e}")
                            continue

                        try:
                            model_fold.fit(X_train_fold, y_train_fold)
                        except Exception as e:
                            print(f"Error training {clf_name} in Fold {fold}: {e}.")
                            continue

                        # Compute evaluation metrics for the validation fold
                        try:
                            y_val_pred = model_fold.predict(X_val_fold)
                            val_accuracy = accuracy_score(y_val_fold, y_val_pred)
                            val_precision = precision_score(y_val_fold, y_val_pred, average='weighted', zero_division=0)
                            val_recall = recall_score(y_val_fold, y_val_pred, average='weighted', zero_division=0)
                            val_f1 = f1_score(y_val_fold, y_val_pred, average='weighted', zero_division=0)
                            val_auc, val_specificity = compute_auc_and_specificity(model_fold, X_val_fold, y_val_fold)
                        except Exception as e:
                            print(f"Error computing validation metrics in Fold {fold} for {clf_name}: {e}")
                            val_accuracy = np.nan
                            val_precision = np.nan
                            val_recall = np.nan
                            val_f1 = np.nan
                            val_auc = np.nan
                            val_specificity = np.nan
                        try:
                            y_test_pred_fold = model_fold.predict(X_test_selected)

                            # compare against TRUE test labels
                            test_accuracy  = accuracy_score(y_test, y_test_pred_fold)
                            test_precision = precision_score(y_test, y_test_pred_fold, average='weighted', zero_division=0)
                            test_recall    = recall_score(y_test, y_test_pred_fold, average='weighted', zero_division=0)
                            test_f1        = f1_score(y_test, y_test_pred_fold, average='weighted', zero_division=0)
                            test_auc, test_specificity = compute_auc_and_specificity(
                                model_fold, X_test_selected, y_test
                            )
                        except Exception as e:
                            print(f"Error computing test metrics in Fold {fold} for {clf_name}: {e}")
                            test_accuracy = np.nan
                            test_precision = np.nan
                            test_recall = np.nan
                            test_f1 = np.nan
                            test_auc = np.nan
                            test_specificity = np.nan


                        # Save fold results in a dictionary
                        result_dict = {
                            'Outcome Classifier': outcome_clf_name,
                            'Feature Selector': fs_name,
                            'Classifier_Analysis': clf_name,
                            'CV_Fold': fold,
                            'Validation_Accuracy': val_accuracy,
                            'Validation_Precision': val_precision,
                            'Validation_Recall': val_recall,
                            'Validation_F1': val_f1,
                            'Validation_AUC': val_auc,
                            'Validation_Specificity': val_specificity,
                            'Test_Accuracy': test_accuracy,
                            'Test_Precision': test_precision,
                            'Test_Recall': test_recall,
                            'Test_F1': test_f1,
                            'Test_AUC': test_auc,
                            'Test_Specificity': test_specificity
                        }
                        fold_results.append(result_dict)

                    # Convert the fold results to a DataFrame and append to the Excel results file
                    df_fold_results = pd.DataFrame(fold_results)
                    append_df_to_excel(results_excel_path, df_fold_results, sheet_name="CV_Results")

                    # Compute the mean metrics across folds and append these aggregated results
                    if not df_fold_results.empty:
                        metrics_cols = [
                            'Validation_Accuracy', 'Validation_Precision', 'Validation_Recall', 'Validation_F1',
                            'Validation_AUC', 'Validation_Specificity', 'Test_Accuracy', 'Test_Precision',
                            'Test_Recall', 'Test_F1', 'Test_AUC', 'Test_Specificity'
                        ]
                        aggregated_mean = df_fold_results[metrics_cols].mean().to_dict()
                        aggregated_mean['Outcome Classifier'] = outcome_clf_name
                        aggregated_mean['Feature Selector'] = fs_name
                        aggregated_mean['Classifier_Analysis'] = clf_name
                        append_df_to_excel(results_excel_path, pd.DataFrame([aggregated_mean]), sheet_name="Aggregated_Results")

                        aggregated_std = df_fold_results[metrics_cols].std().to_dict()
                        aggregated_std['Outcome Classifier'] = outcome_clf_name
                        aggregated_std['Feature Selector'] = fs_name
                        aggregated_std['Classifier_Analysis'] = clf_name

                        append_df_to_excel(results_excel_path, pd.DataFrame([aggregated_std]), sheet_name="Aggregated_Std_Results")


            best_parameters_all.append({
                'Outcome Classifier': outcome_clf_name,
                'Best Parameters': best_params_record
            })

        # Save the final selected features into the results file
        df_selected_features = pd.DataFrame(selected_features_all)
        append_df_to_excel(results_excel_path, df_selected_features, sheet_name="Selected_Features")

        # Save the best parameters for each classifier and selector combination
        best_params_records = []
        for record in best_parameters_all:
            oc = record['Outcome Classifier']
            for fs, clf_dict in record['Best Parameters'].items():
                for clf, params in clf_dict.items():
                    flat_params = {f"{clf}_{k}": v for k, v in params.items()}
                    flat_params['Outcome Classifier'] = oc
                    flat_params['Feature Selector'] = fs
                    flat_params['Classifier'] = clf
                    best_params_records.append(flat_params)
        df_best_params = pd.DataFrame(best_params_records)
        append_df_to_excel(results_excel_path, df_best_params, sheet_name="Best_Parameters")

        # Copy the original dataset file next to the results file for reference.
        original_data_filename = os.path.basename(FILE_PATH)
        destination_path = os.path.join(os.path.dirname(results_excel_path), f"Original_{original_data_filename}")
        shutil.copyfile(FILE_PATH, destination_path)

    except Exception as e:
        print(f"❌ Error processing dataset {dataset_file}: {e}")
        traceback.print_exc()
        continue

print("Pipeline execution completed.")
